In [ ]:
!pip install astroquery
!pip install astroquery pillow tqdm --quiet
!pip install ipywidgets astroquery pillow tqdm --quiet


In [ ]:
#!/usr/bin/env python3
# ===============================================
#  Standalone SDSS Downloader (No CSV Required)
#  Author: Pardis • Advisor: Prof. Safari
# ===============================================
"""
A completely independent script to download images from SDSS.
It randomly selects sky coordinates, fetches images, and saves
them as PNG, FITS, or both — user-configurable via prompts.
"""

import os, gc
import numpy as np
from tqdm import tqdm
from PIL import Image

from astroquery.sdss import SDSS
from astropy import units as u
from astropy.coordinates import SkyCoord


# ========== PROMPTS ==========
print("🌀 Welcome to the SDSS Image Downloader")
n = int(input("🔢 How many images do you want to download? (e.g. 20): "))
fmt = int(input("📦 Save format → 1=PNG | 2=FITS | 3=Both: "))
dest = int(input("📂 Storage → 1=Colab /content | 2=Google Drive | 3=Custom local: "))
folder = input("📁 Folder name or full path (e.g. sdss_images): ").strip()
size = int(input("📐 Image size in pixels (e.g. 200): "))
band = input("🔬 SDSS Band (g/r/i/z) [default: r]: ").strip().lower() or "r"

# ========== STORAGE PATH ==========
if dest == 1:
    path = f"/content/{folder}"
elif dest == 2:
    from google.colab import drive
    drive.mount('/content/drive')
    path = f"/content/drive/MyDrive/{folder}"
else:
    path = folder
os.makedirs(path, exist_ok=True)
print(f"\n✅ Saving images to: {path}")
print(f"📡 Fetching {n} sky positions with SDSS Band '{band}'")

# ========== NORMALIZATION FUNCTION ==========
def normalize_png(data, size):
    norm = (data - data.min()) / np.ptp(data)
    return Image.fromarray(np.uint8(255 * norm)).convert("L").resize((size, size))

# ========== MAIN DOWNLOAD LOOP ==========
success = 0
for i in tqdm(range(n), desc="Downloading"):
    ra = np.random.uniform(0, 360)
    dec = np.random.uniform(-1, 1) * np.random.uniform(0, 90)
    try:
        coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame="icrs")
        imgs = SDSS.get_images(coordinates=coord, band=band)
        if not imgs:
            continue
        hdul = imgs[0]
        data = hdul[0].data.astype(float)

        if fmt in (2, 3):
            fits_name = os.path.join(path, f"gal_{i:04d}.fits")
            hdul.writeto(fits_name, overwrite=True)
        if fmt in (1, 3):
            png_name = os.path.join(path, f"gal_{i:04d}.png")
            normalize_png(data, size).save(png_name)

        del data, hdul
        gc.collect()
        success += 1
    except Exception as e:
        print(f"⚠️ Error at RA={ra:.2f}, DEC={dec:.2f}: {e}")

print(f"\n✅ Done. Downloaded {success}/{n} images successfully.")
print(f"📁 Stored at: {path}")
